# Lab 2: Model Identification



*Your Name:* 

Fitting models to data is an engineering skill that links between the real world of engineering systems to the theory you've been learning in the classroom. For this laboratory session you will collect data from a step test experiments, then fit the data to models derived from first-principles energy balances. 

## Spring 2025 TODO List
* Move the Step Tests and Data Collection to Lab 1.
* Rename from Lab 3 to Lab 2.
* Change the pre-lab exercise to performing nonlinear regression on your Lab 1 data.
* Switch this lab to a sine wave test.
* Extend the lab to include parameter estimation with both sets of data.
* Remove the one-state model? Or keep it but streamline the code to be the same as the two-state model?

## Procedure

1. Download a copy of this notebook to your laptop and complete the the data collection exercises shown below. The results should be embedded in the notebook. Be sure to 'save-as-you-go' to avoid losing your work. 

2. Use your step test data to fit two versions of a model for the temperature control lab:
  * One-state model with one input.
  * Two-state model with one input.

3. Submit your completed lab notebook via Gradescope.

## Exercise 0. Fitting a First-Order Linear Model

As discussed in class, a simple energy balance model for T1 is given by

$$C_p \frac{dT_1}{dt} = U_a(T_{amb} - T_1) + \alpha P_1 U_1$$

where the parameter $\alpha$ has, through independent means, been determined as 0.00016 when U1 is given in percent of full power and power is measured in Watts.

Do the following:
1. Perform nonlinear regression to estimate $U_a$ and $C_p$. In Lab 1, we estimated $\tau$ and $K$ and then calculated $U_a$ and $C_p$. In this lab, you will instead directly regress $U_a$ and $C_p$.
2. Quantify the uncertainty in the estimates $U_a$ and $C_p$ by computing the Fisher information matrix, covariance matrix, and correlation matrix.
3. Perform an eigendecomposition of the covariance matrix.

This pre-lab exercise builds up the analysis workflow you will adapt for the rest of the lab.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Start by loading your data from the previous lab

# Add your solution here

# Next, quickly plot your data from the previous lab

# Add your solution here

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

# Next complete the function below which simulates 
# the first order model for the temperature of the heater.

def tclab_model1(data, param, alpha = 0.00016, P1 = 200, plot=False):
    """ First order linear model for TCLab
    
    Arguments:
        data: pandas dataframe with columns 'Time', 'T1', 'Q1'
        param: list of parameters [Ua, CpH]
        alpha: power conversion factor, default 0.00016 watts / (units P1 * percent U1)
        P1: power setting for heater 1, default 200 (P1 units)
    
    Returns:
        T1: numpy array of model predictions for T1
    """

    # unpack the adjustable parameters
    Ua, CpH = param

    # extract ambient temperature
    T_amb = data['T1'][0]

    # extract experiment time
    t_expt = data['Time']

    # model solution
    def deriv(t, y):

        # interpolate the heater input to nearest value is faster than linear
        u = np.interp(t, data['Time'], data['Q1'], t)

        # unpack the state
        T1 = y

        # Add your solution here


        return dT1dt
    soln = solve_ivp(deriv, [min(t_expt), max(t_expt)], [T_amb, T_amb], t_eval=t_expt)

    T1 = soln.y[0]

    if plot:
        # Plot the temperature data and heat power
        plt.figure()
        plt.subplot(2,1,1)
        plt.plot(t_expt, data['T1'], 'ro', label='Measured')
        plt.plot(t_expt, T1, 'b-', label='Predicted')
        plt.ylabel('Temperature (degC)')
        plt.legend()
        plt.subplot(2,1,2)
        plt.plot(t_expt, data['Q1'], 'r-', label='Heater')
        plt.ylabel('Heater (%)')
        plt.legend()
        plt.xlabel('Time (sec)')
        plt.tight_layout()
        plt.show()

    return T1


# Test your function with the data and initial parameters
# Use the initial parameters from Lab 1 for Ua and CpH
# Add your solution here


In [ ]:
from scipy.optimize import least_squares

# Next, complete the function below to perform nonlinear regression

def tclab_estimation1(data, param_guess=[0.1, 4], plot=True):
    """ Nonlinear regression for TCLab one-state model
    
    Arguments:
        data: pandas dataframe with columns 'Time', 'T1', 'Q1'
        param_guess: list of initial guess for parameters [Ua, CpH]
    
    Returns:
        results: scipy least_squares results object
        residuals: numpy array of residuals
    """
    
    def residual(param):
        # Add your solution here

    # Set bounds (non-negative)
    bnds = ([0.0, 0.0], [np.inf, np.inf])

    results = least_squares(residual, param_guess, verbose=2, method='trf', bounds=bnds, loss='arctan')

    if plot:
        tclab_model1(data, results.x, plot=True)

    residuals = tclab_model1(data, results.x) - data['T1']

    if plot:
        # Plot the residuals
        plt.figure()
        plt.hist(residuals)

        # font size
        fs = 20
        
        # labels
        plt.xlabel("$T_1$ residuals [$^\circ{}$C]",fontsize=fs,fontweight = 'bold')
        plt.ylabel("Count",fontsize=fs,fontweight = 'bold')

        # define tick size
        plt.xticks(fontsize=fs)
        plt.yticks(fontsize=fs)
        plt.tick_params(direction="in",top=True, right=True)

        # finish plot
        plt.show()

    return results, residuals

# Test your function with the data
step_test_results1, step_test_residuals1 = tclab_estimation1(data1)

def print_results1(results):
    print(f"Ua = {results.x[0]:1.3f} W/degC")
    print(f"CpH = {results.x[1]:1.3f} J/degC")
    print(f"Success: {results.success}")
    print(f"Message: {results.message}")

# Save the results for later
Ua_ex0 = step_test_results1.x[0]
Cp_ex0 = step_test_results1.x[1]

print_results1(step_test_results1)

TODO: Add explanation here to review FIM, covariance, ...

In [ ]:
# Perform uncertainty quantification

def covariance_to_correlation(cov):
    ''' Convert covariance matrix into correlation matrix

    Argument:
        cov: covariance matrix

    Returns:
        cor: correlation matrix

    '''

    # Copy matrix
    cor = cov.copy()

    # Get number of rows
    n = cor.shape[0]

    # Loop over rows
    for r in range(n):
        # Loop over columns
        for c in range(n):
            # Scale element
            cor[r,c] = cor[r,c] / np.sqrt(cov[r,r]*cov[c,c])

    return cor


def uncertainty_quantification(results, residuals):
    """ Uncertainty quantification for the estimated parameters
    
    Arguments:
        res: scipy least_squares results object
    
    Returns:
        fim: Fisher Information Matrix
        cov: covariance matrix
    """

    # compute standard deviation of the residuals
    sigma_residuals = np.sqrt(residuals.T @ residuals / (len(residuals) - len(results.x)))
    print("Standard deviation of the residuals = ", round(sigma_residuals, 3), " deg C")
    print("")

    # compute Fisher Information Matrix
    # Add your solution here
    print("Fisher Information Matrix")
    print(fim)
    print("")

    # compute the covariance matrix
    print("Covariance Matrix")
    # Add your solution here
    print(cov)
    print("")

    # eigendecomposition of the covariance matrix
    print("Eigendecomposition of the covariance matrix")
    # Add your solution here

    print("")
    for i in range(len(w)):
        print(f"lambda_{i+1} = {w[i]:1.3e}")
        print(f"v_{i+1} = {v[:,i]}")
        print("")


    # compute the correlation matrix
    print("Correlation Matrix")
    cor = covariance_to_correlation(cov)
    print(cor)
    
    return fim

fim1 = uncertainty_quantification(step_test_results1, step_test_residuals1)

TODO: Add discussion question.

## Exercise 1. Sine Test and Data Collection

### Step 1. Verify operation.

Execute the following cell to verify that you have a working connection to the temperature control lab hardware. This will test for installation of TCLab.py, connection to the Arduino device, and working firmware within the Arduino.

In [ ]:
from tclab import TCLab, clock, Historian, Plotter, setup

run_tclab = False

"""
In the labs, we will us "run_tclab" to control whether the TCLab is used.
After you finish the lab experiment, set run_tclab = False.
This way, you can run all the cells without losing your TCLab output.
"""

if run_tclab:

    TCLab = setup(connected=True)

    lab = TCLab()
    print("TCLab Temperatures:", lab.T1, lab.T2)
    lab.close()

### Step 2.  Check for steady state

As discussed in class, for step testing the device must be initially at steady state. Run the following code to verify the heaters are off and that the temperatures are at a steady ambient temperature.

In [ ]:
if run_tclab:
    # experimental parameters
    tfinal = 30

    # perform experiment
    with TCLab() as lab:
        lab.U1 = 0
        lab.U2 = 0
        h = Historian(lab.sources)
        p = Plotter(h, tfinal)
        for t in clock(tfinal):
            p.update(t)

### Step 3. Sine Test

The step test consists of turning one heater one at 50% power and recording temperature data for at least 1200 seconds. Copy the code from Step 2 into the following cell. Then modify as needed to accomplish the step test with P1 at 200 and using 50% of maximum power.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def sine_wave(t, period=4*60, amplitude=50, offset=50):
    """ Generate a sine wave with the given parameters.

    Arguments:
    t:          time (seconds)
    period:     period of the sine wave (seconds)
    amplitude:  amplitude of the sine wave
    offset:     offset of the sine wave
    
    """
    return amplitude * np.sin(2 * np.pi * t / period) + offset

t = np.linspace(0, 900, 901)
plt.plot(t, sine_wave(t))
plt.xlabel('Time / seconds')
plt.ylabel('Power / %')
plt.show()



In [ ]:
if run_tclab:

# Add your solution here

### Step 4. Save data to a .csv file

Run the following cell to verify and save your data to a '.csv' file. Be sure you can find and locate the data on your laptop before ending your session. You will need access to this data for subsequent exercises.

In [ ]:
import matplotlib.pyplot as plt
import os.path

# Change the filename here
data_file = 'lab2-sine-test.csv'

if run_tclab:

    # Set to True to overwrite the file. Default is False
    # to safeguard against accidentally rerunning this cell.
    overwrite_file = False

    if not overwrite_file and os.path.isfile('./'+data_file):
        raise FileExistsError(data_file + ' already exisits. Either choose a new filename or set overwrite_file = True.')
    elif run_tclab:
        h.to_csv(data_file)
        print("Successfully saved data to "+data_file)
    else:
        print("Data was not saved because run_tclab = False")

## Exercise 2. Fit First-Order Model using Sine Test Data

You will now repeat the steps for Exercise 0 but using the sine test data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load the data
# Add your solution here

# Add your solution here


In [ ]:
# Simulate the first order model for the sine wave test
# using the values of Ua and Cp you identified in Exercise 0

# Add your solution here

In [ ]:
# Perform nonlinear regression on the sine wave test data
# Add your solution here

# Save the results for later
Ua_ex1 = sine_test_results1.x[0]
Cp_ex1 = sine_test_results1.x[1]

print_results1(sine_test_results1)
### END SOLUTION

In [ ]:
# Perform uncertainty quantification on the sine wave test data
# Add your solution here

## Exercise 3. Fitting a Second-Order Model using Sine Test Data

A second order model is for the heater/sensor combination is given by

$$
\begin{align}
C^H_p\frac{dT_{H,1}}{dt} & = U_a(T_{amb} - T_{H,1}) + U_b(T_{S,1} - T_{H,1}) + P_1u_1\\
C^S_p\frac{dT_{S,1}}{dt} & = U_b(T_{H,1} - T_{S,1}) 
\end{align}
$$

where $T_{H,1}$ is the heater temperature, $T_{S,1}$ is the sensor temperature, and $U_b$ is the heat transfer coefficient between the heater and sensor. 

Modify the code you developed for Exercises 0 and 2 to fit this second order model.

Do the following:
1. Report your best fit for $U_a$, $U_b$, $C^H_p$, and $C^S_p$ with units and a reasonable number of significant digits.
2. Characterize the uncertainty in these estimates.
3. Visualize the quality of fit and the residuals.

In [ ]:
def tclab_model2(data, param, alpha = 0.00016, P1 = 200, plot=False):
    """ Second order linear model for TCLab
    
    Arguments:
        data: pandas dataframe with columns 'Time', 'T1', 'Q1'
        param: list of parameters [Ua, Ub, CpH, CpS]
        alpha: power conversion factor, default 0.00016 watts / (units P1 * percent U1)
        P1: power setting for heater 1, default 200 (P1 units)
    
    Returns:
        TS: numpy array of model predictions for TS
    """

    # Add your solution here

    return TS

TS = tclab_model2(data2, [0.1, 0.2, 4, 0.1], plot=True)

In [ ]:
# Next, complete the function below to perform nonlinear regression

def tclab_estimation2(data, param_guess=[0.1, 0.2, 4, 0.1], plot=True):
    """ Nonlinear regression for TCLab two-state model
    
    Arguments:
        data: pandas dataframe with columns 'Time', 'T1', 'Q1'
        param_guess: list of initial guess for parameters [Ua, Ub, CpH, CpS]
    
    Returns:
        results: scipy least_squares results object
        residuals: numpy array of residuals
    """
    
    # Add your solution here

    return results, residuals

# Test your function with the data
sine_test_results2, sine_test_residuals2 = tclab_estimation2(data2)

def print_results2(results):
    print(f"Ua = {results.x[0]:1.3f} W/degC")
    print(f"Ub = {results.x[1]:1.3f} W/degC")
    print(f"CpH = {results.x[2]:1.3f} J/degC")
    print(f"CpS = {results.x[3]:1.3f} J/degC")
    print(f"Success: {results.success}")
    print(f"Message: {results.message}")

# Save the results for later
Ua_ex3, Ub_ex3, CpH_ex3, CpS_ex3 = sine_test_results2.x

print_results2(sine_test_results2)

In [ ]:
# Perform uncertainty quantification
# Add your solution here


## Exercise 4: Fit Second Order Modeling Using Step Test Data

In [ ]:
# Perform regression
# Add your solution here

In [ ]:
# Perform uncertainty quantification
# Add your solution here

## Exercise 5: Fit Second Order Model Using Both Data Sets

In [ ]:
# Modify the tclab_estimation2 function to support multiple datasets

## Exercise 6: Analyze Uncertainty Estimates for Second-Order Models

## Exercise 7: Estimate Heat and Sesnor Time Constants

In [ ]:
# Calculate and print model parameters with uncertainty

# Add your solution here

print("The value of Ua is",round(Ua,4), "+/-", round(np.sqrt(cov_p[0,0]),4), "Watts/degC.")
print("The value of Cp is", round(Cp,3), "+/-", round(np.sqrt(cov_p[1,1]),3), "Joules/degC.")
print("The value of the time constant (tau) is", round(tau,3), "+/-", round(np.sqrt(var_tau),3), "seconds.")
print("The value of the steady state gain (K) is",round(K,3),"+/-", round(np.sqrt(var_K),3), "Watts.")

**Discussion Question:** What are the time constants for the heater and sensor?

In [ ]:
# Calculate and print model parameters with uncertainty

# Add your solution here

print("The value of Ua is", round(Ua,4), "+/-", round(np.sqrt(cov_p[0,0]),4), "Watts/degC.")
print("The value of Ub is", round(Ub,4), "+/-", round(np.sqrt(cov_p[1,1]),4), "Watts/degC.")
print("The value of CpH is", round(CpH,3), "+/-", round(np.sqrt(cov_p[2,2]),3), "Joules/degC.")
print("The value of CpS is", round(CpS,3), "+/-", round(np.sqrt(cov_p[3,3]),3), "Joules/degC.")

print("The value of the time constant TH1 is", round(TH1,3), "+/-", round(np.sqrt(cov_tau[0,0]),3), "seconds.")
print("The value of the time constant TS1 is", round(TS1,3), "+/-", round(np.sqrt(cov_tau[1,1]),3), "seconds.")
print("The value of the time constant TH1 + TS1 is", round(addition,3), "+/-", round(np.sqrt(cov_tau[2,2]),3), "seconds.")

**Discussion Question:** Do you see any relationship between the heater and sensor time constants and what you found for the first-order model?

*Answer*: 

## Declarations

**Collaboration**: If you worked with any classmates, please give their names here. Describe the nature of the collaboration.

**Generative AI**: If you used any Generative AI tools, please elaborate here.

**Reminder:** The written discussions responses must be in your own words. Many of these questions ask about your specific results or are open-ended questions with many reasonable answers. Thus we expect unique responses, analyses, and ideas.

We may use writing analysis software to check for overly similar written responses. You are responsible for reviewing the colaboration policy outlined in the class syllabus to avoid violations of the honor code.